In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [2]:
batch_size = 256
num_workers= 4
mnist_train = torchvision.datasets.FashionMNIST(root='Datasets',
                                    train = True, download=True, transform=torchvision.transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='Datasets',
                                    train=False,download=True,transform=torchvision.transforms.ToTensor())

train_iter = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=True,num_workers=num_workers)

In [31]:
num_inputs = 784
num_hiddens = 256
num_outputs = 10

#def init_params():  #3.12看到的
#    w = torch.randn((num_inputs, 1), requires_grad=True)
#    b = torch.zeros(1, requires_grad=True)
#    return [w, b]

#W1 = torch.randn(num_inputs,num_hiddens)
W1 = torch.Tensor(np.random.normal(0,0.01,(num_inputs,num_hiddens)))
b1 = torch.zeros(num_hiddens)
#W2 = torch.randn(num_hiddens,num_outputs)
W2 = torch.Tensor(np.random.normal(0,0.01,(num_hiddens,num_outputs)))
b2 = torch.zeros(num_outputs)
params =[W1,b1,W2,b2]
for param in params:
    param.requires_grad_(requires_grad=True)  #param!!!

def softmax(y):
    y_exp = y.exp()
    return y_exp/y_exp.sum(dim=1,keepdim=True)
    
def relu(X):
    return torch.max(X,torch.tensor(0.0))
test1 = torch.Tensor([-1,2,3])
print(relu(test1))
def net(X):
    hidden = relu(torch.mm(X.view(-1,num_inputs),W1)+b1)
    return torch.mm(hidden,W2)+b2  #这里为什么不能加softmax???!!!加入softmax两种loss函数都得不到什么结果

def cross_entropy_loss(y_hat,y):
    return -torch.log(y_hat.gather(1,y.view(-1,1)))

loss_data = torch.tensor([[1.0,2.0,3],[4,5,6]])
indices = torch.LongTensor([1,2])
#print(cross_entropy_loss(loss_data,indices))
    
def accuracy(y_hat,y):
    return  (y_hat.argmax(dim=1)==y).float().sum().item()  #这里写成mean()导致问题。
def accuracy_data_iter(data_iter):
    acc_sum,n=0.0,0
    for X,y in data_iter:
        y_hat = net(X)
        acc_sum += accuracy(y_hat,y)
        n += y.shape[0]
    return acc_sum/n

def sgd(params,lr,batch_size):  #更新的时候为什么 除以256？？？
    for param in params:
        param.data-=param.grad*lr/batch_size

    
def test(x):
    x.data=x+1   #x=x+1,就不会改变
y = torch.tensor([1,2,3])
test(y)
print(y)
    

tensor([0., 2., 3.])
tensor([2, 3, 4])


In [32]:
epochs =10
lr = 100  #自己写的loss 10000才能勉强能从0.1到0.17
#loss = cross_entropy_loss  #自己写的已经很难训练了
loss = torch.nn.CrossEntropyLoss() #相同情况下，这个loss在随机初始化时很快能寻到0.84+
#在上面参数训的很差的情况下也能很快从0.1训练到0.7+
params =[W1,b1,W2,b2]

for epoch in range(epochs):
    loss_sum,acc_sum,n=0.0,0.0,0
    for X,y in train_iter:
        #print(X.size())
        y_hat = net(X)
        #print((y_hat==0).sum())   #没有0元素
        #test = loss(y_hat,y)
        #print((test==0).sum())
        l = loss(y_hat,y).sum()
        l.backward()
        sgd(params,lr,batch_size)
        for param in params:
            param.grad.data.zero_()
        loss_sum +=l
        acc_sum +=accuracy(y_hat,y)
        n += y.shape[0]
    test_acc = accuracy_data_iter(test_iter)
    print('epoch= %d, loss = %.3f, train accuray = %.3f, test accracy = %.3f'%
         (epoch+1,loss_sum/n,acc_sum/n,test_acc))
        
        

epoch= 1, loss = 0.003, train accuray = 0.711, test accracy = 0.787
epoch= 2, loss = 0.002, train accuray = 0.822, test accracy = 0.739
epoch= 3, loss = 0.002, train accuray = 0.844, test accracy = 0.815
epoch= 4, loss = 0.002, train accuray = 0.855, test accracy = 0.839
epoch= 5, loss = 0.001, train accuray = 0.866, test accracy = 0.835
epoch= 6, loss = 0.001, train accuray = 0.869, test accracy = 0.841
epoch= 7, loss = 0.001, train accuray = 0.877, test accracy = 0.846
epoch= 8, loss = 0.001, train accuray = 0.879, test accracy = 0.856
epoch= 9, loss = 0.001, train accuray = 0.883, test accracy = 0.862
epoch= 10, loss = 0.001, train accuray = 0.887, test accracy = 0.864
